In [4]:
from img2net import *
import matplotlib.pyplot as plt
import networkx as nx
from graphinference import *
from skimage import measure

In [5]:
#largest connected component
def graph_metrics(G):
    # calculate the normalized average degree
    degrees = [d for n, d in G.degree()]
    #avg_degree = sum(degrees) / len(degrees)
    
    # Get the number of components
    num_components = nx.number_connected_components(G)

    #print("Number of components:", num_components)
    
    # Perform hierarchical clustering using Girvan-Newman algorithm
    clusters = nx.algorithms.community.girvan_newman(G)

    # Get the number of clusters
    num_clusters = len(list(clusters))

    #print("Number of clusters:", num_clusters)
    
    # Calculate the biggest component
    biggest_component = max(nx.connected_components(G), key=len)
    
    # Create a subgraph of the biggest component
    subgraph = G.subgraph(biggest_component)
    
        # Compute eccentricity for all nodes
    eccentricities = nx.eccentricity(subgraph)

    # Print the eccentricity of each node
    #for node, eccentricity in eccentricities.items():
        #print(f"Eccentricity of node {node}: {eccentricity}")
    
    # Find the shortest path length
    shortest_path_length = nx.average_shortest_path_length(subgraph)


    avg_cc = nx.average_clustering(subgraph)

    
    return degrees, num_components, num_clusters, eccentricities,  avg_cc

In [6]:
def convert_pixel_to_um(pixel_measurement):
    # calculate the micrometer measurement
    scale_factor=5.86 #m
    magnification=5
    micrometer_measurement = pixel_measurement*(( scale_factor / magnification) ** 2)
    return micrometer_measurement

# Save adj

In [13]:
import numpy as np

test_path = "/Users/behnaz/NCADD/prediction_cell_neurite/data/3SUBDIV"#/sub/sub0"
cell_model_path = "DHM_Cell.hdf5"
neurite_model_path = "DHM_Neurite.hdf5"
patchsize = 128
overlap = 0.2
# Get a list of all .tif files in the test path
file_list = [f for f in os.listdir(test_path) if f[-3:] == "tif"]
print(file_list)

# Calculate graph metrics and append the values to the arrays for each image
for i in range(len(file_list)):
    
    # Extract the ID from the file name and append the file to the appropriate group
    id = file_list[i][:-4]
    print(id,i)
    
    # Read the image
    img = img_read(test_path, file_list[i])
    
    cell_mask, neurite_mask, cell_num = predict_and_segment(img, cell_model_path, neurite_model_path, patchsize, overlap)
   
    vertices, weights, paths, paths_matrix = infer_structural_graph(cell_mask, neurite_mask, find_paths=True)
    adj_mat = weights > 0

    # Save adj_mat as an npy file with the name of the current file_list item
    np.save(f"{id}.npy", adj_mat)


['2022_08_15_D16_D18_CS6.tif', '2022_08_15_D16_D18_CS3.tif', '2022_08_09_D12_CS2_MAX_Rec_BS.tif', '2022_08_11_D08_CS5_5X.tif', '2022_10_07_D16_D15_CS3.tif', '2022_08_19_D08_CS5_5X.tif', '2022_06_28_D12_CS1_7_5X.tif', '2022_05_30_D16_D18_CS1.tif', '2022_08_19_D08_CS1_5X.tif', '2022_08_19_D08_CS3_5X.tif', '2022_08_15_D16_D18_CS5_MAX_Rec_BS.tif', '2022_05_30_D16_D18_CS5.tif', '2022_08_11_D08_CS3_5X.tif', '2022_08_09_D12_CS6_MAX_Rec_BS.tif', '2022_08_15_D16_D18_CS2_MAX_Rec_BS.tif', '2022_10_07_D16_D15_CS5_MAX_Rec_BS.tif', '2023_03_28_D12_CS5_R3_MAX_Rec_BS.tif', '2023_03_28_D12_CS3_R1_MAX_Rec_BS.tif', '2023_03_28_D12_CS1_R2_MAX_Rec_BS.tif', '2022_08_11_D08_CS6_5X.tif', '2022_08_19_D08_CS4_5X.tif', '2023_03_28_D12_CS4_R3_MAX_Rec_BS.tif', '2023_03_28_D12_CS2_R1_MAX_Rec_BS.tif', '2022_08_11_D08_CS4_5X.tif', '2022_11_28_D16_D18_CS2_6f.tif', '2023_03_28_D12_CS1_R3_MAX_Rec_BS.tif', '2022_10_07_D16_D15_CS4_MAX_Rec_BS.tif', '2023_03_28_D12_CS3_R2_MAX_Rec_BS.tif', '2022_10_07_D16_D15_CS1_MAX_Rec_BS.

In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

test_path = "/Users/behnaz/NCADD/prediction_cell_neurite/data/3SUBDIV"#/sub/sub0"
cell_model_path = "DHM_Cell.hdf5"
neurite_model_path = "DHM_Neurite.hdf5"
patchsize = 128
overlap = 0.2

# Create arrays to store the calculated values
degree_arr = []
num_components_arr = []
num_clusters_arr = []
eccentricities_arr = []
avg_cc_arr = []
cell_watershed = []
cell_areas = []
group_data_neurite_length = {}


# Create a dictionary to group the images based on their ID
groups = {'D08':[],'D12':[],'D16':[]}

# Get a list of all .tif files in the test path
file_list = [f for f in os.listdir(test_path) if f[-3:] == "tif"]
print(file_list)

# Calculate graph metrics and append the values to the arrays for each image
for i in range(len(file_list)):
    
    # Extract the ID from the file name and append the file to the appropriate group
    id = file_list[i].split('_')[3]
    print(id,i)
    if id in groups:
        groups[id].append(file_list[i])
    # Read the image
    img = img_read(test_path, file_list[i])
    
    cell_mask, neurite_mask, cell_num = predict_and_segment(img, cell_model_path, neurite_model_path, patchsize, overlap)
    
    cell_watershed.append(cell_num)
    
    # Compute regions from binary mask
    regions = measure.regionprops(label_image=cell_mask)
    cell_areas.append([convert_pixel_to_um(region.area) for region in regions])
    

    vertices, weights, paths, paths_matrix = infer_structural_graph(cell_mask, neurite_mask, find_paths = True)
    adj_mat = weights>0
    G = nx.Graph(adj_mat)
    #compute neurite length
    neurite_length = np.reciprocal(weights)
    neurite_length[np.isinf(neurite_length)] = 0
    up_neurite_length = np.triu(neurite_length,1)
    non_zero_mask = up_neurite_length != 0
    neurite_length = up_neurite_length[non_zero_mask]
    print(type(neurite_length))
    # Append the neurite length to the group data dictionary
    if id in group_data_neurite_length:
        group_data_neurite_length[id].extend(neurite_length)
    else:
        group_data_neurite_length[id] = list(neurite_length)


    
    # Calculate graph metrics for the current image
    degrees, num_components, num_clusters, eccentricities,  avg_cc = graph_metrics(G)
    print(type(eccentricities))
    avg_cc_arr.append(avg_cc)
    # Append the calculated values to the arrays
    degree_arr.append(degrees)
    num_components_arr.append(num_components)
    num_clusters_arr.append(num_clusters)
    # Save eccentricity values in the dictionary
    eccentricities_arr.append(eccentricities)

     

['2022_08_15_D16_D18_CS6.tif', '2022_08_15_D16_D18_CS3.tif', '2022_08_09_D12_CS2_MAX_Rec_BS.tif', '2022_08_11_D08_CS5_5X.tif', '2022_10_07_D16_D15_CS3.tif', '2022_08_19_D08_CS5_5X.tif', '2022_06_28_D12_CS1_7_5X.tif', '2022_05_30_D16_D18_CS1.tif', '2022_08_19_D08_CS1_5X.tif', '2022_08_19_D08_CS3_5X.tif', '2022_08_15_D16_D18_CS5_MAX_Rec_BS.tif', '2022_05_30_D16_D18_CS5.tif', '2022_08_11_D08_CS3_5X.tif', '2022_08_09_D12_CS6_MAX_Rec_BS.tif', '2022_08_15_D16_D18_CS2_MAX_Rec_BS.tif', '2022_10_07_D16_D15_CS5_MAX_Rec_BS.tif', '2023_03_28_D12_CS5_R3_MAX_Rec_BS.tif', '2023_03_28_D12_CS3_R1_MAX_Rec_BS.tif', '2023_03_28_D12_CS1_R2_MAX_Rec_BS.tif', '2022_08_11_D08_CS6_5X.tif', '2022_08_19_D08_CS4_5X.tif', '2023_03_28_D12_CS4_R3_MAX_Rec_BS.tif', '2023_03_28_D12_CS2_R1_MAX_Rec_BS.tif', '2022_08_11_D08_CS4_5X.tif', '2022_11_28_D16_D18_CS2_6f.tif', '2023_03_28_D12_CS1_R3_MAX_Rec_BS.tif', '2022_10_07_D16_D15_CS4_MAX_Rec_BS.tif', '2023_03_28_D12_CS3_R2_MAX_Rec_BS.tif', '2022_10_07_D16_D15_CS1_MAX_Rec_BS.

/Users/behnaz/NCADD/prediction_cell_neurite/Prediction_pipeline_Cell+Neurite/img2net.py:99: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  c_results = cell_model.predict_generator(test_gene, num_test_images, verbose=1)
2023-05-18 14:45:00.991398: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


64/64 [==============================] - 3s 39ms/step


/Users/behnaz/NCADD/prediction_cell_neurite/Prediction_pipeline_Cell+Neurite/img2net.py:108: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  n_results = neurite_model.predict_generator(test_gene, num_test_images, verbose=1)


64/64 [==============================] - 3s 38ms/step


...

<class 'numpy.ndarray'>
<class 'dict'>
D16 1


/Users/behnaz/NCADD/prediction_cell_neurite/Prediction_pipeline_Cell+Neurite/img2net.py:99: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  c_results = cell_model.predict_generator(test_gene, num_test_images, verbose=1)


64/64 [==============================] - 3s 40ms/step


/Users/behnaz/NCADD/prediction_cell_neurite/Prediction_pipeline_Cell+Neurite/img2net.py:108: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  n_results = neurite_model.predict_generator(test_gene, num_test_images, verbose=1)


64/64 [==============================] - 3s 39ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D12 2
64/64 [==============================] - 3s 38ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D08 3
64/64 [==============================] - 3s 37ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D16 4
64/64 [==============================] - 3s 38ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D08 5
64/64 [==============================] - 3s 38ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D12 6
64/64 [==============================] - 3s 40ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D16 7
64/64 [==============================] - 3s 38ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D08 8
64/64 [==============================] - 3s 39ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D08 9
64/64 [==============================] - 3s 40ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D16 10
64/64 [==============================] - 3s 40ms/step
<class 'numpy.ndarray'>
<class 'dict'>
D16 11
64/64 [=

In [9]:

# Create a dictionary to store the arrays for each group
group_data_eccentricities_arr = {group: [] for group in groups}



# Iterate through the files and append the corresponding array value to the group data
for i in range(len(eccentricities_arr)):
    file = file_list[i]
    id = file.split('_')[3]
    group_data_eccentricities_arr[id].append(eccentricities_arr[i])
group_data_eccentricities_arr.keys()

dict_keys(['D08', 'D12', 'D16'])

In [10]:
import json
with open('group_data_eccentricities_arr.json', 'w') as file:
    json.dump(group_data_eccentricities_arr, file) 

In [ ]:
# degree_arr = []
# num_components_arr = []
# num_clusters_arr = []
# avg_cc_arr = []
# cell_watershed = []
# cell_areas = []
# group_data_neurite_length = {}

In [13]:


# Create a dictionary to store the arrays for each group
group_data_degree_arr = {group: [] for group in groups}
group_data_num_components_arr = {group: [] for group in groups}
group_data_num_clusters_arr = {group: [] for group in groups}
group_data_avg_cc_arr = {group: [] for group in groups}
group_data_cell_watershed = {group: [] for group in groups}
group_data_cell_areas = {group: [] for group in groups}


# Iterate through the files and append the corresponding array value to the group data
for i in range(len(degree_arr)):
    file = file_list[i]
    id = file.split('_')[3]
    group_data_degree_arr[id].append(degree_arr[i])
    group_data_num_components_arr[id].append(num_components_arr[i])
    group_data_num_clusters_arr[id].append(num_clusters_arr[i])
    group_data_avg_cc_arr[id].append(avg_cc_arr[i])
    group_data_cell_watershed[id].append(cell_watershed[i])
    group_data_cell_areas[id].append(cell_areas[i])

# Saving dictionaries

In [15]:
import json
# Save the dictionary to a file
with open('group_data_neurite_length.json', 'w') as file:
    json.dump(group_data_neurite_length, file)
    
with open('group_data_degree_arr.json', 'w') as file:
    json.dump(group_data_degree_arr, file)
    
with open('group_data_num_components_arr.json', 'w') as file:
    json.dump(group_data_num_components_arr, file)
    
with open('group_data_num_clusters_arr.json', 'w') as file:
    json.dump(group_data_num_clusters_arr, file)   
    
with open('group_data_avg_cc_arr.json', 'w') as file:
    json.dump(group_data_avg_cc_arr, file)   
    
# with open('group_data_cell_watershed.json', 'w') as file:
#     json.dump(group_data_cell_watershed, file)   
    
with open('group_data_cell_areas.json', 'w') as file:
    json.dump(group_data_cell_areas, file) 

In [16]:
import json
import numpy as np

# Convert the NumPy data types to Python data types
def convert_np_types(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")



# Save the dictionary to a file
with open('group_data_cell_watershed.json', 'w') as file:
    json.dump(group_data_cell_watershed, file, default=convert_np_types)